In [ ]:
!pip install contractions==0.1.73
!pip install fuzzywuzzy==0.18.0
!pip install python-Levenshtein==0.12.2
!pip install sumy
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
import pandas as pd
import string
import unicodedata
import html
import contractions
from fuzzywuzzy import fuzz
from tqdm import tqdm
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp311-cp311-linux_x86_64.whl size=168020 sha256=ebe7541c349c0853077e35c9cbf2c3733c8c2b183eb486dfd0b5be85156bd36f
  Stored in directory: /root/.cache/pip/wheels/08/39/17/367da5b72aee88c27ba574660010188efcad074dc4d02060ed
Successfully built python-Levenshtein
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.2 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=66d6370555a512decef59aacc69d35fad7cb3073d79d8fe

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
quora = pd.read_csv("/content/drive/MyDrive/quora_data.csv")
amazon = pd.read_csv("/content/drive/MyDrive/amazon_data.csv")
stack = pd.read_csv("/content/drive/MyDrive/stackexgange_data.csv")

In [ ]:
quora = quora[["question1", "question2"]]
quora = quora.rename(columns={"question1" : "Question", "question2" : "Answer"})

In [ ]:
stack = stack[["question", "answer"]]
stack = stack.rename(columns={"question" : "Question", "answer" : "Answer"})

In [ ]:
amazon = amazon.rename(columns={"query" : "Question", "pos" : "Answer"})

In [ ]:
quora["source"] = "quora"
amazon["source"] = "amazon"
stack["source"] = "stack"

df = pd.concat([amazon, stack], ignore_index=True)
df = df.sample(frac=1, random_state=42)
df = df.reset_index(drop=True)


In [ ]:


    def drop_duplicats(df, column_name, threshold=80):
      duplicate_rows = []
      for i, row1 in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        is_duplicate = False
        for row2 in duplicate_rows:
          similarity = fuzz.ratio(row1[column_name], row2[column_name])
          if similarity >=  threshold:
            is_duplicate = True
            break
        if not is_duplicate:
          unique_rows.append(row1)

      return pd.DataFrame(unique_rows)


    def preprocess_text(text):
      text = text.lower()
      text = contractions.fix(text)
      text = unicodedata.normalize("NFKD", text)
      text = html.unescape(text)
      text = "".join(char for char in text if char.isalnum() or char.isspace() or char in ['.', ',', '?'])
      text = ' '.join(text.split())
      return text

    def summarize_text(text):
      parser = PlaintextParser.from_string(text, Tokenizer("english"))
      summarizer = TextRankSummarizer()

      if len(text.split()) < 100:  # return the original text if less than 100 words
        return text
      elif len(text.split()) <= 200:  # summarize if between 100 and 200 words (inclusive)
        summary = summarizer(parser.document, 3)
        text = "".join([str(sentence) for sentence in summary])
      elif len(text.split()) <= 300:  # summarize if between 201 and 300 words (inclusive)
        summary = summarizer(parser.document, 3)
        text = "".join([str(sentence) for sentence in summary])

      return text  # return the original text if more than 300 words

In [ ]:
df['Question'] = df['Question'].apply(preprocess_text)
df['Answer'] = df['Answer'].apply(preprocess_text)

In [ ]:
df["word_count"] = df["Answer"].apply(lambda text: len(text.split()))
df = df[df['word_count'] <= 300]
df = df.drop("word_count", axis=1)


In [ ]:
pd.set_option("display.max_colwidth", None)
tqdm.pandas()  # Enable progress_apply for pandas

df["Summary"] = df["Answer"].progress_apply(summarize_text)

100%|██████████| 30451/30451 [18:29<00:00, 27.44it/s]


In [ ]:
df["Answer"] = df["Summary"]
df = df.drop("Summary", axis=1)

In [ ]:
df.sample(5)

,Question,Answer,source
33396,"in the james bond movie the man with the golden gun, the bad guy scaramanga has a dwarf manservantaccomplice named nick nack, played by herve villechaize of later fantasy island fame. late in the movie, when bond is hunting down scaramanga at scaramangas beautiful island mansion, nick nack seems to be on bonds side, wanting scaramanga dead like when nick nack says to bond if you kill him, all this will be mine, and gestures to the island mansion. but after bond does kill scaramanga, nick nack seems angry and tries to kill bond, failing miserably and getting himself strung up on the mast of a boat as a result. late in the man with the golden gun, why did nick nack seem to switch from encouraging bond to avenging scaramanga?","nick nack never intends to help bond. he just wants to lure bond into the funhouse, where scaramanga is waiting to kill him. if i remember correctly, the movie opens with an assassin hired by nick nack going into the funhouse where he is stalked and killed by scaramanga as practice. it seems to be part of nick nacks job. the scene is here.",stack
11028,"i always thought that switzerland was neither a monarchy nor a republic. the members of the federal council are not here by inheritance, nor are they elected by the people. instead, they are elected by the parliament. this system has been recently confirmed, as there was recently a vote where 76.3 of voters wanted to keep the current system, and did not want to elect the government themselves. the main argument in favour of the current system is that it prevents both political extremists and rich rockstars to getting access to the government. instead, only competent politicians with moderate positions and that already distinguished themselves within the parliament can get votes form the majority of the parliament. so the question is does this system make the swiss confederation a defacto republic, or not?","sometimes it is more a technicality than a matter of political relevance, like for the president of the united states, sometimes it is an important aspect of political life, as in the case of parliamentary republics.the particularity of switzerland is the directorial system, that is to say the collegial rule of the federal council, more than the way they are elected.in practical terms nowadays the term republic means anything that is not a monarchy, from the united states of america to the islamic republic of iran.",stack
26640,"you can show me, or you can give me away, but you never can touch me. you can find me between the family, or between strangers when they first met. i can save lives, or destroy them. even a city and a day were named after me. what am i? please explain each line with this hint it should be very easy this is also just a second way to solve this riddle once lincoln was on the way to the ocean to find an antitoxin for his sick brother victor. just before the arrival he got caught by pirates. but they were not attentive enough, so the escape was very easy.","the answer is love based on the hint it is the capitalised letters in lincolnoceanvictoreasy you can show me, or you can give me away, but you never can touch me.can show or give love, but it is intangible you can find me between the family, obvious enough or between strangers when they first met.as pointed out by admiraljota philadlphia is literally the city of brotherly love from the greek philos, love or friendship, and adelphos, brother til.",stack
6261,can this item be mounted on a wall?,maybe. the back of the breadbox is thin plywood that is spot stapled to the frame. it may or may not take the weight. if it was reinforced first with additional fasteners it would probably work.,amazon
15360,the hometic version only works really well in a particular molded plastic chair...will this one work in any style chair?,"i have tried the shiatsu in the recliner, a regular office chair, and on a bar stool with a back.but it works okay in a recliner if you 

In [ ]:
df.to_csv('/content/drive/MyDrive/first_model_data1.csv', index=None)

In [ ]:
def read_csv(csv_file):
  df = pd.read_csv(csv_file)

  return df


def preprocess_text(df):
  def _preprocess(text):
        text = text.lower()
        text = contractions.fix(text)
        text = unicodedata.normalize("NFKD", text)
        text = html.unescape(text)
        text = "".join(char for char in text if char.isalnum() or char.isspace() or char in ['.', ',', '?'])
        text = ' '.join(text.split())
        return text
  df['question'] = df['question'].apply(preprocess)
  df['answer'] = df['answer'].apply(preprocess)

  return df

def summarize_text(df):
    def _summarize(text):
      parser = PlaintextParser.from_string(text, Tokenizer("english"))
      summarizer = TextRankSummarizer()

      if len(text.split()) < 100:
        return text
      elif len(text.split()) <= 200:
        summary = summarizer(parser.document, 3)
        return "".join([str(sentence) for sentence in summary])
      elif len(text.split()) <= 300:
        summary = summarizer(parser.document, 5)
        return "".join([str(sentence) for sentence in summary])
      return text

    df['Summary'] = df['answer'].apply(_summarize)
    df['answer'] = df["Summary"]
    df = df.drop("Summary", axis=1)

    return df

def embedd_questions(df):

  faq_questions = df["question"].tolist()
  faq_answers = df["answer"].tolist()

  embeddings = model.encode(faq_questions)
  embeddings = np.array(embeddings, dtype="float32")

  index = faiss.IndexFlatL2(embeddings.shape[1])
  index.add(embeddings)

  faiss.write_index(index, "/content/drive/MyDrive/faq_fiass_index.bin")

def save_file(df):

  df.to_csv("processed_faqs.csv", index=None)


def preprocess_faqs(csv_file):
  try:
    df = read_csv(csv_file)
    df["word_count"] = df["Answer"].apply(lambda text: len(text.split()))
    df = df[df['word_count'] <= 300]
    df = df.drop("word_count", axis=1)
    df = preprocess_text(df)
    df = summarize_text(df)
    embedd_questions(df)
    save_file(df)
  except Exception as e:
   raise ValueError(f"An error occured while preprocessing FAQs: {e}")






